In [1]:
import simpler_env

In [5]:
print(len(simpler_env.ENVIRONMENTS))
for env in simpler_env.ENVIRONMENTS:
    print(env) 

25
google_robot_pick_coke_can
google_robot_pick_horizontal_coke_can
google_robot_pick_vertical_coke_can
google_robot_pick_standing_coke_can
google_robot_pick_object
google_robot_move_near_v0
google_robot_move_near_v1
google_robot_move_near
google_robot_open_drawer
google_robot_open_top_drawer
google_robot_open_middle_drawer
google_robot_open_bottom_drawer
google_robot_close_drawer
google_robot_close_top_drawer
google_robot_close_middle_drawer
google_robot_close_bottom_drawer
google_robot_place_in_closed_drawer
google_robot_place_in_closed_top_drawer
google_robot_place_in_closed_middle_drawer
google_robot_place_in_closed_bottom_drawer
google_robot_place_apple_in_closed_top_drawer
widowx_spoon_on_towel
widowx_carrot_on_plate
widowx_stack_cube
widowx_put_eggplant_in_basket


In [2]:
env = simpler_env.make('google_robot_pick_coke_can')
obs, reset_info = env.reset()

[2025-03-07 14:55:06.686] [svulkan2] [error] GLFW error: X11: The DISPLAY environment variable is missing
[2025-03-07 14:55:06.686] [svulkan2] [warning] Continue without GLFW.


In [11]:
import pprint
#print(pprint.pprint(obs))

def print_keys_recursively(d, indent=0):
    prefix = "  " * indent  # Compute the indentation prefix once.
    for key, value in d.items():
        if isinstance(value, dict):
            print(f"{prefix} {key}")
            print_keys_recursively(value, indent + 1)
        elif hasattr(value, 'shape'):
            print(prefix + f"{key} : {type(value)} with shape {value.shape}")
        else:
            print(prefix + f"{key} : {type(value)} = {value}")
print_keys_recursively(obs)

 agent
  qpos : <class 'numpy.ndarray'> with shape (11,)
  qvel : <class 'numpy.ndarray'> with shape (11,)
   controller
     gripper
      target_qpos : <class 'numpy.ndarray'> with shape (2,)
  base_pose : <class 'numpy.ndarray'> with shape (7,)
 extra
  tcp_pose : <class 'numpy.ndarray'> with shape (7,)
 camera_param
   base_camera
    extrinsic_cv : <class 'numpy.ndarray'> with shape (4, 4)
    cam2world_gl : <class 'numpy.ndarray'> with shape (4, 4)
    intrinsic_cv : <class 'numpy.ndarray'> with shape (3, 3)
   overhead_camera
    extrinsic_cv : <class 'numpy.ndarray'> with shape (4, 4)
    cam2world_gl : <class 'numpy.ndarray'> with shape (4, 4)
    intrinsic_cv : <class 'numpy.ndarray'> with shape (3, 3)
 image
   base_camera
    rgb : <class 'numpy.ndarray'> with shape (128, 128, 3)
    depth : <class 'numpy.ndarray'> with shape (128, 128, 1)
    Segmentation : <class 'numpy.ndarray'> with shape (128, 128, 4)
   overhead_camera
    rgb : <class 'numpy.ndarray'> with shape (512

In [9]:
print(obs.keys())

odict_keys(['agent', 'extra', 'camera_param', 'image'])


In [1]:
policies = ["rt_1_tf_trained_for_000400120",
            "rt_1_tf_trained_for_000058240",
            "rt_1_x_tf_trained_for_002272480_step",
            "rt_1_tf_trained_for_000001120",
            "octo-base"
            ]

policies_short = ["rt-1-converged",
            "rt-1-15pct",
            "rt-1-x",
            "rt-1-begin",
            "octo-base",
            ]

urdf_versions = ["recolor_cabinet_visual_matching_1", 
                 "recolor_tabletop_visual_matching_1", 
                 "recolor_tabletop_visual_matching_2", 
                 "None"
                 ]

drawer_tasks = ["OpenTopDrawerCustomInScene-v0",
                "OpenMiddleDrawerCustomInScene-v0",
                "OpenBottomDrawerCustomInScene-v0",
                "CloseTopDrawerCustomInScene-v0",
                "CloseMiddleDrawerCustomInScene-v0",
                "CloseBottomDrawerCustomInScene-v0"
                ]

pick_tasks = ["GraspSingleOpenedCokeCanInScene-v0"]

move_tasks = ["MoveNearGoogleBakedTexInScene-v0"]

place_tasks = ["PlaceIntoClosedTopDrawerCustomInScene-v0"]

orientations = ["lr_switch_True","laid_vertically_True", "upright_True"]

drawer_real_success = {
        "open": {
          #  "rt-2-x": 0.333,
            "rt-1-converged": 0.815,
            "rt-1-15pct": 0.704,
            "rt-1-x": 0.519,
            "rt-1-begin": 0.000,
            "octo-base": 0.148,
        },
        "close": {
          #   "rt-2-x": 0.630,
            "rt-1-converged": 0.926,
            "rt-1-15pct": 0.889,
            "rt-1-x": 0.741,
            "rt-1-begin": 0.000,
            "octo-base": 0.519,
        },
    }


num_trials_per_real_task={
    "google_robot_open_drawer" : 27,
    "google_robot_close_drawer" : 27,
    "google_robot_drawer_avg" : 54, # 9 robot locations x 3 shelves (top, middle, bottom) x 2 close/open
    "google_robot_pick_coke_can_horizontally_laying" : 25,
    "google_robot_pick_coke_can_vertically_laying" : 25,
    "google_robot_pick_coke_can_standing" : 25,
    "google_robot_pick_coke_can" : 75, # 25 locations of coke can x 3 coke can orientation (vertically/horizontally laying, standing)
    "google_robot_move_near" : 60, # 5 triplets of objects * 6 combination * 2 triangle patterns
    "google_robot_place_apple_in_closed_top_drawer" : 27, # 3 robot locations x 9 apple locations
}


REAL_PERF = {    # Real robot eval performance --> extract via: REAL_PERF[task][policy]
    "google_robot_pick_coke_can": {
        #"rt-2-x": 0.907,
        "rt-1-converged": 0.853,
        "rt-1-15pct": 0.920,
        "rt-1-x": 0.760,
        "rt-1-begin": 0.133,
        "octo-base": 0.293,
    },
     "google_robot_pick_coke_can_horizontally_laying": {
        #"rt-2-x": 0.920,
        "rt-1-converged": 0.960,
        "rt-1-15pct": 1,
        "rt-1-x": 0.880,
        "rt-1-begin": 0.200,
        "octo-base": 0.440,
    },
     "google_robot_pick_coke_can_vertically_laying": {
        #"rt-2-x": 0.8,
        "rt-1-converged": 0.880,
        "rt-1-15pct": 0.960,
        "rt-1-x": 0.560,
        "rt-1-begin": 0,
        "octo-base": 0.200,
    },
     "google_robot_pick_coke_can_standing": {
        #"rt-2-x": 1,
        "rt-1-converged": 0.720,
        "rt-1-15pct": 0.800,
        "rt-1-x": 0.840,
        "rt-1-begin": 0.200,
        "octo-base": 0.240,
    },
    "google_robot_move_near": {
       # "rt-2-x": 0.733,
        "rt-1-converged": 0.633,
        "rt-1-15pct": 0.583,
        "rt-1-x": 0.450,
        "rt-1-begin": 0.017,
        "octo-base": 0.350,
    },
    "google_robot_drawer_avg": {
      'rt-1-converged': 0.870, 
      'rt-1-15pct': 0.796, 
      'rt-1-x': 0.630, 
      'rt-1-begin': 0.000, 
      'octo-base': 0.333
    },
    "google_robot_open_drawer": {
       # "rt-2-x": 0.333,
        "rt-1-converged": 0.815,
        "rt-1-15pct": 0.704,
        "rt-1-x": 0.519,
        "rt-1-begin": 0.000,
        "octo-base": 0.148,
    },
    "google_robot_close_drawer": {
        #"rt-2-x": 0.630,
        "rt-1-converged": 0.926,
        "rt-1-15pct": 0.889,
        "rt-1-x": 0.741,
        "rt-1-begin": 0.000,
        "octo-base": 0.519,
    },
    "google_robot_place_apple_in_closed_top_drawer": {
        #"rt-2-x": 0.074,
        "rt-1-converged": 0.185,
        "rt-1-15pct": 0.185,
        "rt-1-x": 0.407,
        "rt-1-begin": 0.000,
        "octo-base": 0.000,
    },
    "widowx_spoon_on_towel": {
        "rt-1-x": 0.000,
        "octo-base": 0.333,
        "octo-small": 0.417,
    },
    "widowx_carrot_on_plate": {
        "rt-1-x": 0.000,
        "octo-base": 0.250,
        "octo-small": 0.083,
    },
    "widowx_stack_cube": {
        "rt-1-x": 0.000,
        "octo-base": 0.000,
        "octo-small": 0.125,
    },
    "widowx_put_eggplant_in_basket": {
        "rt-1-x": 0.000,
        "octo-base": 0.250,
        "octo-small": 0.400,
    },
}

In [ ]:
#Alternatively compute the average for the drawer tasks
# Yields slightly different results then the ones reported in the paper (they round down when .5 in the paper)
# Compute the average for each key by iterating over the keys in "open"
#REAL_PERF["google_robot_drawer_avg"] = {
#    key: (REAL_PERF["google_robot_open_drawer"][key] + REAL_PERF["google_robot_close_drawer"][key]) / 2
#    for key in REAL_PERF["google_robot_open_drawer"]
#}

{'rt-1-converged': 0.8705, 'rt-1-15pct': 0.7965, 'rt-1-x': 0.63, 'rt-1-begin': 0.0, 'octo-base': 0.3335}


In [2]:
from rich.tree import Tree
from rich.console import Console
import glob

dir_name = "/nas/longleaf/home/tis/mmai/results"
filenames = glob.glob(f"{dir_name}/**/*.mp4", recursive=True)

def count_files(filenames, extra_pattern_require):
    filtered_filenames = list(filter(lambda filename: all(
        pattern in filename for pattern in extra_pattern_require), filenames))
    return len(filtered_filenames), filtered_filenames

def add_tree_nodes(branch, levels, extra_pattern):
    # for leaf nodes
    if len(levels) == 1:  
        for item in levels[0]:
            # count the files that match the extra pattern 
            result, _ = count_files(filenames, extra_pattern_require=extra_pattern + [item])
            branch.add(f"{result} {item}")
        return
    # call recursively for the next level
    for item in levels[0]:
        sub_branch = branch.add(item)
        add_tree_nodes(sub_branch, levels[1:], extra_pattern + [item])

def print_overview(levels):
    console = Console()
    tree = Tree("Eval Results")
    add_tree_nodes(tree, levels, [])
    console.print(tree)

In [3]:
outcome = ["success", "failure"]
print_overview([urdf_versions, drawer_tasks, policies])
#print_overview([policies, drawer_tasks, urdf_versions, outcome])

print_overview([urdf_versions, pick_tasks, policies, orientations])

print_overview([urdf_versions, move_tasks, policies])
#move_type = ["moved_correct_obj_False", "moved_correct_obj_True"]
#print_overview([urdf_versions, move_tasks, policies, move_type])

print_overview([urdf_versions, place_tasks, policies])
#robot_position = ["a0", "b0", "c0"]
#print_overview([urdf_versions, place_tasks, policies, robot_position])

Eval Results
├── recolor_cabinet_visual_matching_1
│   ├── OpenTopDrawerCustomInScene-v0
│   │   ├── 9 rt_1_tf_trained_for_000400120
│   │   ├── 9 rt_1_tf_trained_for_000058240
│   │   ├── 9 rt_1_x_tf_trained_for_002272480_step
│   │   ├── 9 rt_1_tf_trained_for_000001120
│   │   └── 9 octo-base
│   ├── OpenMiddleDrawerCustomInScene-v0
│   │   ├── 9 rt_1_tf_trained_for_000400120
│   │   ├── 9 rt_1_tf_trained_for_000058240
│   │   ├── 9 rt_1_x_tf_trained_for_002272480_step
│   │   ├── 9 rt_1_tf_trained_for_000001120
│   │   └── 9 octo-base
│   ├── OpenBottomDrawerCustomInScene-v0
│   │   ├── 9 rt_1_tf_trained_for_000400120
│   │   ├── 9 rt_1_tf_trained_for_000058240
│   │   ├── 9 rt_1_x_tf_trained_for_002272480_step
│   │   ├── 9 rt_1_tf_trained_for_000001120
│   │   └── 9 octo-base
│   ├── CloseTopDrawerCustomInScene-v0
│   │   ├── 9 rt_1_tf_trained_for_000400120
│   │   ├── 9 rt_1_tf_trained_for_000058240
│   │   ├── 9 rt_1_x_tf_trained_for_002272480_step
│   │   ├── 9 rt_1_tf_trained_for_000001120
│   │   └── 9 octo-base
│   ├── CloseMiddleDrawerCustomInScene-v0
│   │   ├── 9 rt_1_tf_trained_for_000400120
│   │   ├── 9 rt_1_tf_trained_for_000058240
│   │   ├── 9 rt_1_x_tf_trained_for_002272480_step
│   │   ├── 9 rt_1_tf_trained_for_000001120
│   │   └── 9 octo-base
│   └── CloseBottomDrawerCustomInScene-v0
│       ├── 9 rt_1_tf_trained_for_000400120
│       ├── 9 rt_1_tf_trained_for_000058240
│       ├── 9 rt_1_x_tf_trained_for_002272480_step
│       ├── 9 rt_1_tf_trained_for_000001120
│       └── 9 octo-base
├── recolor_tabletop_visual_matching_1
│   ├── OpenTopDrawerCustomInScene-v0
│   │   ├── 9 rt_1_tf_trained_for_000400120
│   │   ├── 9 rt_1_tf_trained_for_000058240
│   │   ├── 9 rt_1_x_tf_trained_for_002272480_step
│   │   ├── 9 rt_1_tf_trained_for_000001120
│   │   └── 9 octo-base
│   ├── OpenMiddleDrawerCustomInScene-v0
│   │   ├── 9 rt_1_tf_trained_for_000400120
│   │   ├── 9 rt_1_tf_trained_for_000058240
│   │   ├── 9 rt_1_x_tf_trained_for_002272480_step
│   │   ├── 9 rt_1_tf_trained_for_000001120
│   │   └── 9 octo-base
│   ├── OpenBottomDrawerCustomInScene-v0
│   │   ├── 9 rt_1_tf_trained_for_000400120
│   │   ├── 9 rt_1_tf_trained_for_000058240
│   │   ├── 9 rt_1_x_tf_trained_for_002272480_step
│   │   ├── 9 rt_1_tf_trained_for_000001120
│   │   └── 9 octo-base
│   ├── CloseTopDrawerCustomInScene-v0
│   │   ├── 9 rt_1_tf_trained_for_000400120
│   │   ├── 9 rt_1_tf_trained_for_000058240
│   │   ├── 9 rt_1_x_tf_trained_for_002272480_step
│   │   ├── 9 rt_1_tf_trained_for_000001120
│   │   └── 9 octo-base
│   ├── CloseMiddleDrawerCustomInScene-v0
│   │   ├── 9 rt_1_tf_trained_for_000400120
│   │   ├── 9 rt_1_tf_trained_for_000058240
│   │   ├── 9 rt_1_x_tf_trained_for_002272480_step
│   │   ├── 9 rt_1_tf_trained_for_000001120
│   │   └── 9 octo-base
│   └── CloseBottomDrawerCustomInScene-v0
│       ├── 9 rt_1_tf_trained_for_000400120
│       ├── 9 rt_1_tf_trained_for_000058240
│       ├── 9 rt_1_x_tf_trained_for_002272480_step
│       ├── 9 rt_1_tf_trained_for_000001120
│       └── 9 octo-base
├── recolor_tabletop_visual_matching_2
│   ├── OpenTopDrawerCustomInScene-v0
│   │   ├── 9 rt_1_tf_trained_for_000400120
│   │   ├── 9 rt_1_tf_trained_for_000058240
│   │   ├── 9 rt_1_x_tf_trained_for_002272480_step
│   │   ├── 9 rt_1_tf_trained_for_000001120
│   │   └── 9 octo-base
│   ├── OpenMiddleDrawerCustomInScene-v0
│   │   ├── 9 rt_1_tf_trained_for_000400120
│   │   ├── 9 rt_1_tf_trained_for_000058240
│   │   ├── 9 rt_1_x_tf_trained_for_002272480_step
│   │   ├── 9 rt_1_tf_trained_for_000001120
│   │   └── 9 octo-base
│   ├── OpenBottomDrawerCustomInScene-v0
│   │   ├── 9 rt_1_tf_trained_for_000400120
│   │   ├── 9 rt_1_tf_trained_for_000058240
│   │   ├── 9 rt_1_x_tf_trained_for_002272480_step
│   │   ├── 9 rt_1_tf_trained_for_000001120
│   │   └── 9 octo-base
│   ├── CloseTopDrawerCustomInScene-v0
│   │   ├── 9 rt_1_tf_trained_for_000400120
│   │   ├── 9 rt_1_tf_trained_for_000058240
│   │   ├── 9 rt_1_x_tf_trained_f

Eval Results
├── recolor_cabinet_visual_matching_1
│   └── GraspSingleOpenedCokeCanInScene-v0
│       ├── rt_1_tf_trained_for_000400120
│       │   ├── 25 lr_switch_True
│       │   ├── 25 laid_vertically_True
│       │   └── 25 upright_True
│       ├── rt_1_tf_trained_for_000058240
│       │   ├── 25 lr_switch_True
│       │   ├── 25 laid_vertically_True
│       │   └── 25 upright_True
│       ├── rt_1_x_tf_trained_for_002272480_step
│       │   ├── 25 lr_switch_True
│       │   ├── 25 laid_vertically_True
│       │   └── 25 upright_True
│       ├── rt_1_tf_trained_for_000001120
│       │   ├── 25 lr_switch_True
│       │   ├── 25 laid_vertically_True
│       │   └── 25 upright_True
│       └── octo-base
│           ├── 25 lr_switch_True
│           ├── 25 laid_vertically_True
│           └── 25 upright_True
├── recolor_tabletop_visual_matching_1
│   └── GraspSingleOpenedCokeCanInScene-v0
│       ├── rt_1_tf_trained_for_000400120
│       │   ├── 25 lr_switch_True
│       │   ├── 25 laid_vertically_True
│       │   └── 25 upright_True
│       ├── rt_1_tf_trained_for_000058240
│       │   ├── 25 lr_switch_True
│       │   ├── 25 laid_vertically_True
│       │   └── 25 upright_True
│       ├── rt_1_x_tf_trained_for_002272480_step
│       │   ├── 25 lr_switch_True
│       │   ├── 25 laid_vertically_True
│       │   └── 25 upright_True
│       ├── rt_1_tf_trained_for_000001120
│       │   ├── 25 lr_switch_True
│       │   ├── 25 laid_vertically_True
│       │   └── 25 upright_True
│       └── octo-base
│           ├── 25 lr_switch_True
│           ├── 25 laid_vertically_True
│           └── 25 upright_True
├── recolor_tabletop_visual_matching_2
│   └── GraspSingleOpenedCokeCanInScene-v0
│       ├── rt_1_tf_trained_for_000400120
│       │   ├── 25 lr_switch_True
│       │   ├── 25 laid_vertically_True
│       │   └── 25 upright_True
│       ├── rt_1_tf_trained_for_000058240
│       │   ├── 25 lr_switch_True
│       │   ├── 25 laid_vertically_True
│       │   └── 25 upright_True
│       ├── rt_1_x_tf_trained_for_002272480_step
│       │   ├── 25 lr_switch_True
│       │   ├── 25 laid_vertically_True
│       │   └── 25 upright_True
│       ├── rt_1_tf_trained_for_000001120
│       │   ├── 25 lr_switch_True
│       │   ├── 25 laid_vertically_True
│       │   └── 25 upright_True
│       └── octo-base
│           ├── 25 lr_switch_True
│           ├── 25 laid_vertically_True
│           └── 25 upright_True
└── None
    └── GraspSingleOpenedCokeCanInScene-v0
        ├── rt_1_tf_trained_for_000400120
        │   ├── 25 lr_switch_True
        │   ├── 25 laid_vertically_True
        │   └── 25 upright_True
        ├── rt_1_tf_trained_for_000058240
        │   ├── 25 lr_switch_True
        │   ├── 25 laid_vertically_True
        │   └── 25 upright_True
        ├── rt_1_x_tf_trained_for_002272480_step
        │   ├── 25 lr_switch_True
        │   ├── 25 laid_vertically_True
        │   └── 25 upright_True
        ├── rt_1_tf_trained_for_000001120
        │   ├── 25 lr_switch_True
        │   ├── 25 laid_vertically_True
        │   └── 25 upright_True
        └── octo-base
            ├── 25 lr_switch_True
            ├── 25 laid_vertically_True
            └── 25 upright_True

Eval Results
├── recolor_cabinet_visual_matching_1
│   └── MoveNearGoogleBakedTexInScene-v0
│       ├── 60 rt_1_tf_trained_for_000400120
│       ├── 60 rt_1_tf_trained_for_000058240
│       ├── 60 rt_1_x_tf_trained_for_002272480_step
│       ├── 60 rt_1_tf_trained_for_000001120
│       └── 60 octo-base
├── recolor_tabletop_visual_matching_1
│   └── MoveNearGoogleBakedTexInScene-v0
│       ├── 60 rt_1_tf_trained_for_000400120
│       ├── 60 rt_1_tf_trained_for_000058240
│       ├── 60 rt_1_x_tf_trained_for_002272480_step
│       ├── 60 rt_1_tf_trained_for_000001120
│       └── 60 octo-base
├── recolor_tabletop_visual_matching_2
│   └── MoveNearGoogleBakedTexInScene-v0
│       ├── 60 rt_1_tf_trained_for_000400120
│       ├── 60 rt_1_tf_trained_for_000058240
│       ├── 60 rt_1_x_tf_trained_for_002272480_step
│       ├── 60 rt_1_tf_trained_for_000001120
│       └── 60 octo-base
└── None
    └── MoveNearGoogleBakedTexInScene-v0
        ├── 60 rt_1_tf_trained_for_000400120
        ├── 60 rt_1_tf_trained_for_000058240
        ├── 60 rt_1_x_tf_trained_for_002272480_step
        ├── 60 rt_1_tf_trained_for_000001120
        └── 60 octo-base

Eval Results
├── recolor_cabinet_visual_matching_1
│   └── PlaceIntoClosedTopDrawerCustomInScene-v0
│       ├── 27 rt_1_tf_trained_for_000400120
│       ├── 27 rt_1_tf_trained_for_000058240
│       ├── 27 rt_1_x_tf_trained_for_002272480_step
│       ├── 27 rt_1_tf_trained_for_000001120
│       └── 27 octo-base
├── recolor_tabletop_visual_matching_1
│   └── PlaceIntoClosedTopDrawerCustomInScene-v0
│       ├── 27 rt_1_tf_trained_for_000400120
│       ├── 27 rt_1_tf_trained_for_000058240
│       ├── 27 rt_1_x_tf_trained_for_002272480_step
│       ├── 27 rt_1_tf_trained_for_000001120
│       └── 27 octo-base
├── recolor_tabletop_visual_matching_2
│   └── PlaceIntoClosedTopDrawerCustomInScene-v0
│       ├── 27 rt_1_tf_trained_for_000400120
│       ├── 27 rt_1_tf_trained_for_000058240
│       ├── 27 rt_1_x_tf_trained_for_002272480_step
│       ├── 27 rt_1_tf_trained_for_000001120
│       └── 27 octo-base
└── None
    └── PlaceIntoClosedTopDrawerCustomInScene-v0
        ├── 27 rt_1_tf_trained_for_000400120
        ├── 27 rt_1_tf_trained_for_000058240
        ├── 27 rt_1_x_tf_trained_for_002272480_step
        ├── 27 rt_1_tf_trained_for_000001120
        └── 27 octo-base

In [4]:
from anytree import Node, RenderTree
import re
from collections import Counter

def filter_files(filenames, patterns):
    # Return the count and list of filenames that match all given patterns.
    matches = [f for f in filenames if all(re.search(p, f) for p in patterns)]
    return len(matches), matches

def create_parent_node(name, aggregated, children):
    """
    Given a name, a dictionary of aggregated counts, and a list of child nodes,
    compute the success rate and create a summary Node with attributes success,
    failure, and success_rate. If the sum of counts is zero, return None.
    """
    success = aggregated.get("success", 0)
    failure = aggregated.get("failure", 0)
    total = success + failure
    if total == 0:
        return None
    success_rate = success / total
    # Create the parent node with attributes; the label is just the name.
    parent = Node(name, success=success, failure=failure, success_rate=success_rate)
    for child in children:
        child.parent = parent
    return parent

def build_tree(name, levels, extra_pattern):
    """
    Recursively builds an anytree.Node and returns a tuple (aggregated_counts, node).
    
    - When levels has one list (base case), it creates child nodes for each outcome.
    - For higher levels, it recurses into sub-levels, aggregates counts via Counter,
      and only creates nodes if there is a nonzero count.
      
    The extra_pattern list is built up recursively.
    
    Supports tuple entries: if an item is a tuple, the first element is used for 
    pattern matching and the second element is used as the node name.
    """
    # Base case: last level with outcomes (e.g., "success" and "failure")
    if len(levels) == 1:
        aggregated = {}
        children = []
        for outcome in levels[0]:
            pat, outcome_label = outcome if isinstance(outcome, tuple) else (outcome, outcome)
            count, _ = filter_files(filenames, extra_pattern + [pat])
            aggregated[outcome_label] = count
            if count > 0:
                children.append(Node(f"{count} {outcome_label}"))
        parent_node = create_parent_node(name, aggregated, children)
        return aggregated, parent_node
    else:
        aggregated_total = Counter()
        children = []
        for item in levels[0]:
            pat, node_name = item if isinstance(item, tuple) else (item, item)
            child_aggregated, child_node = build_tree(node_name, levels[1:], extra_pattern + [pat])
            aggregated_total.update(child_aggregated)
            if child_node is not None:
                children.append(child_node)
        parent_node = create_parent_node(name, dict(aggregated_total), children)
        return dict(aggregated_total), parent_node

# Get node by path as pattern list 
def get_node(root, pattern_list):
    if not pattern_list:
        return root  
    current_pattern = pattern_list[0]
    for child in root.children:
        if current_pattern == child.name:
            result = get_node(child, pattern_list[1:])
            if result is not None:
                return result  
    return None  

def get_node(root, pattern_list):
    if not pattern_list:
        return root

    # First, try to match one of the children with the current pattern.
    for child in root.children:
        if child.name == pattern_list[0]:
            result = get_node(child, pattern_list[1:])
            if result is not None:
                return result

    # If no match was found but the current node has exactly one child,
    # try skipping this level (i.e. do not consume the current pattern).
    if len(root.children) == 1:
        result = get_node(root.children[0], pattern_list)
        if result is not None:
            return result

    return None

# Print the entire tree
def print_tree(tree):
    for pre, _, node in RenderTree(tree):
        stats = f"-> success: {node.success}, failure: {
            node.failure}, success_rate: {node.success_rate:.3f}" if node.children else ""
        print(f"{pre}{node.name} {stats}")

In [5]:
outcome = ["success", "failure"]
higher_tasks = [(r"Open(?:Bottom|Top|Middle)Drawer", "google_robot_open_drawer"),
                (r"Close(?:Bottom|Top|Middle)Drawer", "google_robot_close_drawer")]

average_level = [(r"(?:Open|Close)(?:Bottom|Top|Middle)Drawer","google_robot_drawer_avg")]
#average_level = [(r"","google_robot_drawer_avg")]
levels = [policies, average_level, higher_tasks, drawer_tasks, urdf_versions, outcome]
_ , drawer_tree = build_tree("Eval Results for Drawer Tasks", levels, [])
print_tree(drawer_tree)

Eval Results for Drawer Tasks -> success: 476, failure: 604, success_rate: 0.441
├── rt_1_tf_trained_for_000400120 -> success: 154, failure: 62, success_rate: 0.713
│   └── google_robot_drawer_avg -> success: 154, failure: 62, success_rate: 0.713
│       ├── google_robot_open_drawer -> success: 63, failure: 45, success_rate: 0.583
│       │   ├── OpenTopDrawerCustomInScene-v0 -> success: 23, failure: 13, success_rate: 0.639
│       │   │   ├── recolor_cabinet_visual_matching_1 -> success: 6, failure: 3, success_rate: 0.667
│       │   │   │   ├── 6 success 
│       │   │   │   └── 3 failure 
│       │   │   ├── recolor_tabletop_visual_matching_1 -> success: 6, failure: 3, success_rate: 0.667
│       │   │   │   ├── 6 success 
│       │   │   │   └── 3 failure 
│       │   │   ├── recolor_tabletop_visual_matching_2 -> success: 5, failure: 4, success_rate: 0.556
│       │   │   │   ├── 5 success 
│       │   │   │   └── 4 failure 
│       │   │   └── None -> success: 6, failure: 3, succe

In [7]:
orientation_tasks = [("lr_switch_True", "google_robot_pick_coke_can_horizontally_laying"), 
                     ("laid_vertically_True", "google_robot_pick_coke_can_vertically_laying"), 
                     ("upright_True", "google_robot_pick_coke_can_standing")]

#Note tuple for renaming purposes
pick_task= [("GraspSingleOpenedCokeCanInScene-v0", "google_robot_pick_coke_can")]
levels = [policies, pick_task, orientation_tasks, urdf_versions, outcome]
_ , pick_coke_can_tree = build_tree("Eval Results For Pick Coke Can", levels, [])
print_tree(pick_coke_can_tree)

Eval Results For Pick Coke Can -> success: 709, failure: 791, success_rate: 0.473
├── rt_1_tf_trained_for_000400120 -> success: 259, failure: 41, success_rate: 0.863
│   └── google_robot_pick_coke_can -> success: 259, failure: 41, success_rate: 0.863
│       ├── google_robot_pick_coke_can_horizontally_laying -> success: 97, failure: 3, success_rate: 0.970
│       │   ├── recolor_cabinet_visual_matching_1 -> success: 24, failure: 1, success_rate: 0.960
│       │   │   ├── 24 success 
│       │   │   └── 1 failure 
│       │   ├── recolor_tabletop_visual_matching_1 -> success: 24, failure: 1, success_rate: 0.960
│       │   │   ├── 24 success 
│       │   │   └── 1 failure 
│       │   ├── recolor_tabletop_visual_matching_2 -> success: 25, failure: 0, success_rate: 1.000
│       │   │   └── 25 success 
│       │   └── None -> success: 24, failure: 1, success_rate: 0.960
│       │       ├── 24 success 
│       │       └── 1 failure 
│       ├── google_robot_pick_coke_can_vertically_laying

In [8]:
move_task = [("MoveNearGoogleBakedTexInScene-v0", "google_robot_move_near")]
levels = [policies, move_task, urdf_versions, outcome]
_ , move_near_tree = build_tree("Eval Results For Move Near", levels, [])
print_tree(move_near_tree)

Eval Results For Move Near -> success: 291, failure: 909, success_rate: 0.242
├── rt_1_tf_trained_for_000400120 -> success: 98, failure: 142, success_rate: 0.408
│   └── google_robot_move_near -> success: 98, failure: 142, success_rate: 0.408
│       ├── recolor_cabinet_visual_matching_1 -> success: 26, failure: 34, success_rate: 0.433
│       │   ├── 26 success 
│       │   └── 34 failure 
│       ├── recolor_tabletop_visual_matching_1 -> success: 25, failure: 35, success_rate: 0.417
│       │   ├── 25 success 
│       │   └── 35 failure 
│       ├── recolor_tabletop_visual_matching_2 -> success: 25, failure: 35, success_rate: 0.417
│       │   ├── 25 success 
│       │   └── 35 failure 
│       └── None -> success: 22, failure: 38, success_rate: 0.367
│           ├── 22 success 
│           └── 38 failure 
├── rt_1_tf_trained_for_000058240 -> success: 94, failure: 146, success_rate: 0.392
│   └── google_robot_move_near -> success: 94, failure: 146, success_rate: 0.392
│       ├── rec

In [9]:
place_task = [("PlaceIntoClosedTopDrawerCustomInScene-v0", "google_robot_place_apple_in_closed_top_drawer")]
levels = [policies, place_task, urdf_versions, outcome]
_ , place_apple_tree = build_tree("Eval Results For Place Apple in Closed Top Drawer", levels, [])
print_tree(place_apple_tree)

Eval Results For Place Apple in Closed Top Drawer -> success: 53, failure: 487, success_rate: 0.098
├── rt_1_tf_trained_for_000400120 -> success: 9, failure: 99, success_rate: 0.083
│   └── google_robot_place_apple_in_closed_top_drawer -> success: 9, failure: 99, success_rate: 0.083
│       ├── recolor_cabinet_visual_matching_1 -> success: 2, failure: 25, success_rate: 0.074
│       │   ├── 2 success 
│       │   └── 25 failure 
│       ├── recolor_tabletop_visual_matching_1 -> success: 1, failure: 26, success_rate: 0.037
│       │   ├── 1 success 
│       │   └── 26 failure 
│       ├── recolor_tabletop_visual_matching_2 -> success: 4, failure: 23, success_rate: 0.148
│       │   ├── 4 success 
│       │   └── 23 failure 
│       └── None -> success: 2, failure: 25, success_rate: 0.074
│           ├── 2 success 
│           └── 25 failure 
├── rt_1_tf_trained_for_000058240 -> success: 20, failure: 88, success_rate: 0.185
│   └── google_robot_place_apple_in_closed_top_drawer -> success

In [10]:
results = get_results(drawer_tree, ["google_robot_drawer_avg"])
print(results)

NameError: name 'get_results' is not defined

In [11]:
# Define the regex patterns to traverse the tree.
#patterns = ["rt_1_tf_trained_for_000400120", "Open(?:Bottom|Top|Middle)Drawer", "OpenTopDrawerCustomInScene-v0"]
patterns = ["octo-base", "google_robot_open_drawer", "OpenTopDrawerCustomInScene-v0"]

node = get_node(drawer_tree, patterns)
print(node.name, "\nSuccess:", node.success, "\nFailure:", node.failure, "\nSuccess Rate", node.success_rate)
print_tree(node)

OpenTopDrawerCustomInScene-v0 
Success: 0 
Failure: 36 
Success Rate 0.0
OpenTopDrawerCustomInScene-v0 -> success: 0, failure: 36, success_rate: 0.000
├── recolor_cabinet_visual_matching_1 -> success: 0, failure: 9, success_rate: 0.000
│   └── 9 failure 
├── recolor_tabletop_visual_matching_1 -> success: 0, failure: 9, success_rate: 0.000
│   └── 9 failure 
├── recolor_tabletop_visual_matching_2 -> success: 0, failure: 9, success_rate: 0.000
│   └── 9 failure 
└── None -> success: 0, failure: 9, success_rate: 0.000
    └── 9 failure 


In [33]:
import numpy as np
from scipy.stats import kruskal, pearsonr
from typing import Sequence
import pandas as pd 

def mean_maximum_rank_violation(
    perf_sim: Sequence[float], perf_real: Sequence[float]
) -> float:
    x = np.array(perf_sim)
    y = np.array(perf_real)
    assert x.shape == y.shape
    
    # pairwise comparison
    sim_comp = x[:, None] > x[None, :]
    real_comp = y[:, None] > y[None, :]
    
    mask = sim_comp != real_comp
    diff_matrix = np.abs(y[:, None] - y[None, :])
    max_rank_violations = np.max(diff_matrix * mask, axis=1)
    
    return float(np.mean(max_rank_violations))



def get_results(tree, patterns):
    nodes = [get_node(tree, [policy, *patterns]) for policy in policies]
    return np.array([[node.success_rate, node.success+node.failure] for node in nodes]) 

# Takes in a list of sublists [mean, length] 
# and generates trials for each sublist (array of 0 and 1s)
generate_trials = lambda mean_len_list: [
    np.concatenate([np.ones(int(round(m * l)), dtype=int), 
                    np.zeros(int(l - round(m * l)), dtype=int)]).tolist()
    for m, l in mean_len_list
]

n = len(policies_short)
results_df = pd.DataFrame({"Eval Setup": ["Real"] * n + ["Sim"] * n + ["Metric"] * 3, 
                           "Policy": policies_short * 2 + ["MMRV", "Pearson", "#Kruskal p<0.05"]})

def evaluate_task(tree, task):
    real_results = np.array([[s, num_trials_per_real_task[task]] 
                             for s in REAL_PERF[task].values()])
    sim_results = get_results(tree, [task]) 

    # Compute Statistics
    #pearson_corr = pearson_correlation(sim_results[:, 0], real_results[:, 0])
    pearson_corr = pearsonr(sim_results[:, 0], real_results[:, 0])[0]
    
    mmrv = mean_maximum_rank_violation(sim_results[:, 0], real_results[:, 0])
    # Perform the Kruskal-Wallis test for each policy
    kruskal_pvalues = np.array([
        kruskal(sim, real).pvalue if np.mean(sim) != np.mean(real) else 1
        for sim, real in zip(generate_trials(sim_results), generate_trials(real_results))
    ])
    # Count the number of policies with p-value < 0.05
    kruskal_count = np.sum(kruskal_pvalues < 0.05)

    # Store results in the Eval Results DataFrame
    results_df[task] = np.concatenate([real_results[:, 0], sim_results[:, 0], [mmrv, pearson_corr, kruskal_count]])

#Ensure tasks are in num_trials_per_real_task, and real eval results are present 
for task in ["google_robot_pick_coke_can_horizontally_laying", 
             "google_robot_pick_coke_can_vertically_laying", 
             "google_robot_pick_coke_can_standing",
             "google_robot_pick_coke_can"]:
    evaluate_task(pick_coke_can_tree, task)

for task in ["google_robot_move_near"]:
    evaluate_task(move_near_tree, task)

for task in ["google_robot_open_drawer", "google_robot_close_drawer", "google_robot_drawer_avg"]:
    evaluate_task(drawer_tree, task)

for task in ["google_robot_place_apple_in_closed_top_drawer"]:
    evaluate_task(place_apple_tree, task)

new_order = [0, 1, 3, 2, 4, 5, 6, 8, 7, 9, 10, 11, 12]
results_df = results_df.reindex(new_order)

print(results_df.to_string())


   Eval Setup           Policy  google_robot_pick_coke_can_horizontally_laying  google_robot_pick_coke_can_vertically_laying  google_robot_pick_coke_can_standing  google_robot_pick_coke_can  google_robot_move_near  google_robot_open_drawer  google_robot_close_drawer  google_robot_drawer_avg  google_robot_place_apple_in_closed_top_drawer
0        Real   rt-1-converged                                        0.960000                                      0.880000                             0.720000                    0.853000                0.633000                  0.815000                   0.926000                 0.870000                                       0.185000
1        Real       rt-1-15pct                                        1.000000                                      0.960000                             0.800000                    0.920000                0.583000                  0.704000                   0.889000                 0.796000                               

In [13]:
# 11     Metric          Pearson                                        0.983855                                      0.955338                             0.902385                    0.972208                0.869981                  0.978606                   0.868121                 0.957418                                       0.920535
# 11     Metric          Pearson                                        0.983855                                      0.955338                             0.902385                    0.972208                0.869981                  0.978606                   0.868121                 0.957418                                       0.920535

#10     Metric             MMRV                                        0.016000                                      0.032000                             0.064000                    0.026800                0.000000                  0.000000                   0.103600                 0.066400                                       0.000000
#10     Metric             MMRV                                        0.016000                                      0.032000                             0.064000                    0.026800                0.000000                  0.000000                   0.103600                 0.066400                                       0.000000


In [14]:
import pandas as pd

# Define the data with the new policy names and column names
data = {
    "Eval Setup": ["Sim Paper"] * 5 + ["Metric"] * 3,
    "Policy": ["rt-1-converged", "rt-1-15pct", "rt-1-x", "rt-1-begin", "octo-base", "MMRV", "Pearson", "#Kruskal p<0.05"],
    "google_robot_pick_coke_can_horizontally_laying": [0.960, 0.860, 0.820, 0.050, 0.210, 0.027, 0.981, 0],
    "google_robot_pick_coke_can_vertically_laying": [0.900, 0.790, 0.330, 0.000, 0.210, 0.027, 0.964, 0],
    "google_robot_pick_coke_can_standing": [0.710, 0.480, 0.550, 0.030, 0.090, 0.053, 0.942, 2],
    "google_robot_pick_coke_can": [0.857, 0.710, 0.567, 0.027, 0.170, 0.031, 0.976, 3],
    "google_robot_move_near": [0.442, 0.354, 0.317, 0.050, 0.042, 0.111, 0.855, 3],
    "google_robot_open_drawer": [0.601, 0.463, 0.296, 0.000, 0.009, 0.0, 0.983, 1],
    "google_robot_close_drawer": [0.861, 0.667, 0.891, 0.278, 0.444, 0.123, 0.768, 2],
    "google_robot_drawer_avg": [0.730, 0.565, 0.597, 0.139, 0.227, 0.055, 0.915, 2],
    "google_robot_place_apple_in_closed_top_drawer": [0.065, 0.130, 0.213, 0.000, 0.000, 0.0, 0.969, 0],
}



# Create the DataFrame and set 'Policy' as the index
sim_paper_df = pd.DataFrame(data)
new_order = [0, 1, 3, 2, 4, 5, 6, 7]
sim_paper_df = sim_paper_df.reindex(new_order).reset_index(drop=True)
# Display the DataFrame
print(sim_paper_df.to_string())

  Eval Setup           Policy  google_robot_pick_coke_can_horizontally_laying  google_robot_pick_coke_can_vertically_laying  google_robot_pick_coke_can_standing  google_robot_pick_coke_can  google_robot_move_near  google_robot_open_drawer  google_robot_close_drawer  google_robot_drawer_avg  google_robot_place_apple_in_closed_top_drawer
0  Sim Paper   rt-1-converged                                           0.960                                         0.900                                0.710                       0.857                   0.442                     0.601                      0.861                    0.730                                          0.065
1  Sim Paper       rt-1-15pct                                           0.860                                         0.790                                0.480                       0.710                   0.354                     0.463                      0.667                    0.565                                  

In [15]:
import pandas as pd 
sim_results_df = results_df[results_df["Eval Setup"].isin(["Sim", "Metric"])].reset_index(drop=True)
#print(sim_results_df.to_string())
#print(sim_paper_df.to_string())
numeric_cols = sim_results_df.select_dtypes(include='number').columns
results_diff_df = sim_results_df.copy()
results_diff_df[numeric_cols] = sim_results_df[numeric_cols] - sim_paper_df[numeric_cols].reset_index(drop=True)
print(results_diff_df.to_string())

  Eval Setup           Policy  google_robot_pick_coke_can_horizontally_laying  google_robot_pick_coke_can_vertically_laying  google_robot_pick_coke_can_standing  google_robot_pick_coke_can  google_robot_move_near  google_robot_open_drawer  google_robot_close_drawer  google_robot_drawer_avg  google_robot_place_apple_in_closed_top_drawer
0        Sim   rt-1-converged                                        0.010000                                     -0.030000                             0.040000                    0.006333               -0.033667                 -0.017667                  -0.018407                -0.017037                                       0.018333
1        Sim       rt-1-15pct                                        0.040000                                      0.010000                             0.000000                    0.016667                0.037667                 -0.027815                   0.018185                -0.004815                                  

In [16]:
task_renaming ={
    "google_robot_open_drawer": ("Open/Close Drawer", "Open"),
    "google_robot_close_drawer": ("Open/Close Drawer", "Close"),
    "google_robot_drawer_avg": ("Open/Close Drawer", "Average"),
    "google_robot_move_near": ("Move Near", "Average"),
    "google_robot_pick_coke_can_horizontally_laying": ("Pick Coke Can", "Horizontally Laying"),
    "google_robot_pick_coke_can_vertically_laying": ("Pick Coke Can", "Vertically Laying"),
    "google_robot_pick_coke_can_standing": ("Pick Coke Can", "Standing"),
    "google_robot_pick_coke_can": ("Pick Coke Can", "Average"),
    "google_robot_place_apple_in_closed_top_drawer": ("Open Top Drawer and Place Apple", "Average")
}

policy_renaming = {
        "rt-2-x": "RT-2-X",
        "rt-1-converged": "RT-1 (Converged)",
        "rt-1-15pct": "RT-1 (15\%)",
        "rt-1-begin": "RT-1 (Begin)",
        "rt-1-x": "RT-1-X",
        "octo-base": "Octo-Base",
}

def format_df(input_df):
    df = input_df.copy()
    df = df.round(3)
    df = df.rename(columns=task_renaming)
    df['Policy'] = df['Policy'].replace(policy_renaming)
    df = df.set_index(["Eval Setup", "Policy"])
    df.columns = pd.MultiIndex.from_tuples(df.columns)
    return df

#results_df = results_df.round(3)
#results_df['Policy'] = results_df['Policy'].replace(policy_renaming)
#results_df = results_df.set_index(["Eval Setup", "Policy"])
#results_df.columns = pd.MultiIndex.from_tuples(results_df.columns)

<>:16: SyntaxWarning: invalid escape sequence '\%'
<>:16: SyntaxWarning: invalid escape sequence '\%'
/tmp/ipykernel_1569842/2664587387.py:16: SyntaxWarning: invalid escape sequence '\%'
  "rt-1-15pct": "RT-1 (15\%)",


In [17]:
formatted_results_df = format_df(results_df)
display(formatted_results_df)

Pick Coke Can                             \
                            Horizontally Laying Vertically Laying Standing   
Eval Setup Policy                                                            
Real       RT-1 (Converged)               0.960             0.880    0.720   
           RT-1 (15\%)                    1.000             0.960    0.800   
           RT-1 (Begin)                   0.200             0.000    0.200   
           RT-1-X                         0.880             0.560    0.840   
           Octo-Base                      0.440             0.200    0.240   
Sim        RT-1 (Converged)               0.970             0.870    0.750   
           RT-1 (15\%)                    0.900             0.800    0.480   
           RT-1 (Begin)                   0.050             0.000    0.050   
           RT-1-X                         0.820             0.310    0.570   
           Octo-Base                      0.160             0.250    0.110   
Metric     MMRV                           0.016             0.032    0.064   
           Pearson                        0.984             0.955    0.902   
           #Kruskal p<0.05                2.000             1.000    3.000   

                                    Move Near Open/Close Drawer         \
                            Average   Average              Open  Close   
Eval Setup Policy                                                        
Real       RT-1 (Converged)   0.853     0.633             0.815  0.926   
           RT-1 (15\%)        0.920     0.583             0.704  0.889   
           RT-1 (Begin)       0.133     0.017             0.000  0.000   
           RT-1-X             0.760     0.450             0.519  0.741   
           Octo-Base          0.293     0.350             0.148  0.519   
Sim        RT-1 (Converged)   0.863     0.408             0.583  0.843   
           RT-1 (15\%)        0.727     0.392             0.435  0.685   
           RT-1 (Begin)       0.033     0.033             0.000  0.241   
           RT-1-X             0.567     0.338             0.259  0.926   
           Octo-Base          0.173     0.042             0.009  0.426   
Metric     MMRV               0.027     0.000             0.000  0.104   
           Pearson            0.972     0.870             0.979  0.868   
           #Kruskal p<0.05    4.000     3.000             4.000  3.000   

                                    Open Top Drawer and Place Apple  
                            Average                         Average  
Eval Setup Policy                                                    
Real       RT-1 (Converged)   0.870                           0.185  
           RT-1 (15\%)        0.796                           0.185  
           RT-1 (Begin)       0.000                           0.000  
           RT-1-X             0.630                           0.407  
           Octo-Base          0.333                           0.000  
Sim        RT-1 (Converged)   0.713                           0.083  
           RT-1 (15\%)        0.560                           0.185  
           RT-1 (Begin)       0.120                           0.000  
           RT-1-X             0.593                           0.222  
           Octo-Base          0.218                           0.000  
Metric     MMRV               0.066                           0.000  
           Pearson            0.957                           0.921  
           #Kruskal p<0.05    3.000                           0.000

In [18]:
formatted_results_diff_df= format_df(results_diff_df)
display(formatted_results_diff_df)

Pick Coke Can                             \
                            Horizontally Laying Vertically Laying Standing   
Eval Setup Policy                                                            
Sim        RT-1 (Converged)               0.010            -0.030    0.040   
           RT-1 (15\%)                    0.040             0.010    0.000   
           RT-1 (Begin)                   0.000             0.000    0.020   
           RT-1-X                         0.000            -0.020    0.020   
           Octo-Base                     -0.050             0.040    0.020   
Metric     MMRV                          -0.011             0.005    0.011   
           Pearson                        0.003            -0.009   -0.040   
           #Kruskal p<0.05                2.000             1.000    1.000   

                                    Move Near Open/Close Drawer         \
                            Average   Average              Open  Close   
Eval Setup Policy                                                        
Sim        RT-1 (Converged)   0.006    -0.034            -0.018 -0.018   
           RT-1 (15\%)        0.017     0.038            -0.028  0.018   
           RT-1 (Begin)       0.006    -0.017             0.000 -0.037   
           RT-1-X            -0.000     0.021            -0.037  0.035   
           Octo-Base          0.003    -0.000             0.000 -0.018   
Metric     MMRV              -0.004    -0.111             0.000 -0.019   
           Pearson           -0.004     0.015            -0.004  0.100   
           #Kruskal p<0.05    1.000     0.000             3.000  1.000   

                                    Open Top Drawer and Place Apple  
                            Average                         Average  
Eval Setup Policy                                                    
Sim        RT-1 (Converged)  -0.017                           0.018  
           RT-1 (15\%)       -0.005                           0.055  
           RT-1 (Begin)      -0.019                           0.000  
           RT-1-X            -0.004                           0.009  
           Octo-Base         -0.009                           0.000  
Metric     MMRV               0.011                           0.000  
           Pearson            0.042                          -0.048  
           #Kruskal p<0.05    1.000                           0.000

In [19]:
latex_results = formatted_results_df.to_latex(
    float_format=lambda x: f"{x:.3f}"
)
print(latex_results)

\begin{tabular}{llrrrrrrrrr}
\toprule
 &  & \multicolumn{4}{r}{Pick Coke Can} & Move Near & \multicolumn{3}{r}{Open/Close Drawer} & Open Top Drawer and Place Apple \\
 &  & Horizontally Laying & Vertically Laying & Standing & Average & Average & Open & Close & Average & Average \\
Eval Setup & Policy &  &  &  &  &  &  &  &  &  \\
\midrule
\multirow[t]{5}{*}{Real} & RT-1 (Converged) & 0.960 & 0.880 & 0.720 & 0.853 & 0.633 & 0.815 & 0.926 & 0.870 & 0.185 \\
 & RT-1 (15\%) & 1.000 & 0.960 & 0.800 & 0.920 & 0.583 & 0.704 & 0.889 & 0.796 & 0.185 \\
 & RT-1 (Begin) & 0.200 & 0.000 & 0.200 & 0.133 & 0.017 & 0.000 & 0.000 & 0.000 & 0.000 \\
 & RT-1-X & 0.880 & 0.560 & 0.840 & 0.760 & 0.450 & 0.519 & 0.741 & 0.630 & 0.407 \\
 & Octo-Base & 0.440 & 0.200 & 0.240 & 0.293 & 0.350 & 0.148 & 0.519 & 0.333 & 0.000 \\
\cline{1-11}
\multirow[t]{5}{*}{Sim} & RT-1 (Converged) & 0.970 & 0.870 & 0.750 & 0.863 & 0.408 & 0.583 & 0.843 & 0.713 & 0.083 \\
 & RT-1 (15\%) & 0.900 & 0.800 & 0.480 & 0.727 & 0.392 

In [20]:
latex_results = formatted_results_diff_df.to_latex(
    float_format=lambda x: f"{x:.3f}"
)
print(latex_results)

\begin{tabular}{llrrrrrrrrr}
\toprule
 &  & \multicolumn{4}{r}{Pick Coke Can} & Move Near & \multicolumn{3}{r}{Open/Close Drawer} & Open Top Drawer and Place Apple \\
 &  & Horizontally Laying & Vertically Laying & Standing & Average & Average & Open & Close & Average & Average \\
Eval Setup & Policy &  &  &  &  &  &  &  &  &  \\
\midrule
\multirow[t]{5}{*}{Sim} & RT-1 (Converged) & 0.010 & -0.030 & 0.040 & 0.006 & -0.034 & -0.018 & -0.018 & -0.017 & 0.018 \\
 & RT-1 (15\%) & 0.040 & 0.010 & 0.000 & 0.017 & 0.038 & -0.028 & 0.018 & -0.005 & 0.055 \\
 & RT-1 (Begin) & 0.000 & 0.000 & 0.020 & 0.006 & -0.017 & 0.000 & -0.037 & -0.019 & 0.000 \\
 & RT-1-X & 0.000 & -0.020 & 0.020 & -0.000 & 0.021 & -0.037 & 0.035 & -0.004 & 0.009 \\
 & Octo-Base & -0.050 & 0.040 & 0.020 & 0.003 & -0.000 & 0.000 & -0.018 & -0.009 & 0.000 \\
\cline{1-11}
\multirow[t]{3}{*}{Metric} & MMRV & -0.011 & 0.005 & 0.011 & -0.004 & -0.111 & 0.000 & -0.019 & 0.011 & 0.000 \\
 & Pearson & 0.003 & -0.009 & -0.040 & -0.0

In [21]:
results_df.to_csv('eval_results.csv', index=False)

In [34]:
import glob
from pathlib import Path
from typing import Sequence, Optional

import numpy as np



SIMPLER_PERF = {    # SIMPLER simulated eval performance --> extract via: SIMPLER_PERF[task][policy]
    "google_robot_pick_coke_can": {
        "rt-2-x": 0.787,
        "rt-1-converged": 0.857,
        "rt-1-15pct": 0.710,
        "rt-1-x": 0.567,
        "rt-1-begin": 0.027,
        "octo-base": 0.170,
    },
    "google_robot_move_near": {
        "rt-2-x": 0.779,
        "rt-1-converged": 0.442,
        "rt-1-15pct": 0.354,
        "rt-1-x": 0.317,
        "rt-1-begin": 0.050,
        "octo-base": 0.042,
    },
    "google_robot_open_drawer": {
        "rt-2-x": 0.157,
        "rt-1-converged": 0.601,
        "rt-1-15pct": 0.463,
        "rt-1-x": 0.296,
        "rt-1-begin": 0.000,
        "octo-base": 0.009,
    },
    "google_robot_close_drawer": {
        "rt-2-x": 0.343,
        "rt-1-converged": 0.861,
        "rt-1-15pct": 0.667,
        "rt-1-x": 0.891,
        "rt-1-begin": 0.278,
        "octo-base": 0.444,
    },
    "google_robot_place_apple_in_closed_top_drawer": {
        "rt-2-x": 0.037,
        "rt-1-converged": 0.065,
        "rt-1-15pct": 0.130,
        "rt-1-x": 0.213,
        "rt-1-begin": 0.000,
        "octo-base": 0.000,
    },
    "widowx_spoon_on_towel": {
        "rt-1-x": 0.000,
        "octo-base": 0.125,
        "octo-small": 0.472,
    },
    "widowx_carrot_on_plate": {
        "rt-1-x": 0.042,
        "octo-base": 0.083,
        "octo-small": 0.097,
    },
    "widowx_stack_cube": {
        "rt-1-x": 0.000,
        "octo-base": 0.000,
        "octo-small": 0.042,
    },
    "widowx_put_eggplant_in_basket": {
        "rt-1-x": 0.000,
        "octo-base": 0.431,
        "octo-small": 0.569,
    },
}


In [35]:
from scipy.stats import pearsonr
# Test pearsonr for an array with unequal length
try:
    result = pearsonr([1, 2, 3], [1, 2, 3])
    print(type(result[0]))
except ValueError as e:
    print(f"Error for unequal length arrays: {e}")

# Test pearsonr for an array with equal length but only zeros
try:
    result = pearsonr([0, 0, 0], [0, 0, 0])
    print(f"Pearson r for arrays with only zeros: {result[0]}")
except ValueError as e:
    print(f"Error for arrays with only zeros: {e}")
    

<class 'numpy.float64'>
Pearson r for arrays with only zeros: nan


/tmp/ipykernel_1569842/3193595706.py:11: ConstantInputWarning: An input array is constant; the correlation coefficient is not defined.
  result = pearsonr([0, 0, 0], [0, 0, 0])


In [28]:
print(float(np.nan))

nan
